In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics

import sys
sys.path.append("..")
from UNSW_DFv2 import *

In [2]:
train, test = DF_preprocessed_traintest()
X_train, X_test, y_train, y_test = DF_XY()

Reading Preprocessed CSV Files..
	 Train Shape:  	 (175341, 54)
	 Test Shape:  	 (82332, 54)
Dataset Loaded!
( 1 ) Reading Preprocessed CSV files..
	 Training dataset loaded..
	 Testing dataset loaded..

( 2 ) Loading done, splitting into X and Y..
	 ( 2.1 ) x_train Shape:  	 (175341, 53)
	 ( 2.2 ) y_train Shape:  	 (175341,)
	 ( 2.3 ) x_test Shape:  	 (82332, 53)
	 ( 2.4 ) y_test Shape:  	 (82332,)
( 3 ) Done!
PS! Import with: x_train, x_test, y_train, y_test = XY_import()


In [3]:
train.drop(train.index[10000:175341], axis=0, inplace=True)
test.drop(test.index[5000:82332], axis=0, inplace=True)
print(f"Train shape:\t {train.shape}\nTest shape:\t {test.shape}")

Train shape:	 (10000, 54)
Test shape:	 (5000, 54)


In [4]:
error = []
C_value = []
accuracy_score = []
precision_score = []
F1_score = []
recall_score = []

def SVM_predict(c_start, c_end, svm_kernel, svm_gamma, svm_degree):
    """Predicts an SVM model with given arguments

    Args:
        c_start (int): C value start
        c_end (int): C value end
        svm_kernel (string): SVM kernel given in a string format: i.e. linear, poly, rbf
        svm_degree (int): default=3, Degree of the polynomial kernel function (‘poly’). Ignored by all other kernels.
    """
    c_start = c_start
    c_end += 50
    
    for c in range(c_start, c_end):
        SVM_model = SVC(kernel=svm_kernel, C = c, gamma=svm_gamma, degree=svm_degree)
        SVM_model.fit(X_train, y_train)
        pred_i = SVM_model.predict(X_test)
        
        # Appending values to list
        error.append(np.mean(pred_i != y_test))
        C_value.append(c)
        accuracy_score.append(metrics.accuracy_score(y_test, pred_i))
        precision_score.append(metrics.precision_score(y_test, pred_i))
        F1_score.append(metrics.f1_score(y_test, pred_i))
        recall_score.append(metrics.recall_score(y_test, pred_i))
    

In [8]:
# Calling the function created above.
SVM_predict(c_start=1, c_end=200, svm_kernel="rbf", svm_gamma="auto", svm_degree=3)

In [9]:
# Creating a dataframe and saving to file
# dictionary of lists 
dict = {
        'C': C_value, 
        'accuracy': accuracy_score, 
        'precision': precision_score,
        'F1': F1_score,
        'recall': recall_score,
        'error': error
        }

df = pd.DataFrame(dict, index=C_value)
df.set_index("C", inplace = True)
df

,accuracy,precision,F1,recall,error
C,,,,,
1,0.815795,0.751375,0.856020,0.994529,0.184205
2,0.819475,0.755497,0.858392,0.993735,0.180525
3,0.822038,0.758336,0.860073,0.993338,0.177962
4,0.823483,0.760219,0.860943,0.992434,0.176517
5,0.824989,0.762002,0.861910,0.991970,0.175011
...,...,...,...,...,...
196,0.837050,0.786129,0.867305,0.967176,0.162950
197,0.837099,0.786186,0.867339,0.967176,0.162901
198,0.837159,0.786266,0.867380,0.967153,0.162841


In [10]:
# EXPORT AS CSV when done.
df.to_csv('SVM_scores(1-200)_kernel_z.csv')